In [1]:
#function：input:feature，out:its type
#0：SR  1：MR  2：VB  3：VR  4：WR
import torch
from torch import nn
import pandas as pd
import time
import copy
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import xlrd
from torch.autograd import Variable
from torch.utils.data import Dataset  
from torch.utils.data import DataLoader 
from torchvision import datasets
import torch.utils.data as Data
import os

from __future__ import print_function
import torch.optim as optim
import time

In [2]:
#-------------------FUNCTION OF IMPORT 2 DATASET----------------
class lungDataset(Dataset):#此类继承自dataset
    def __init__(self,inputs,sig_num,in_width,in_height):  #
        xy1=np.loadtxt(open(inputs,"rb"),delimiter=",",skiprows=0)
        
        x=np.zeros([sig_num,in_width,in_height]) 
        y=np.zeros([sig_num])
        for i in range(sig_num):  
            x[i,:,:]=xy1[in_height*i:in_height*(i+1),0:256].T
            ###DATA
            y[i]=xy1[in_height*i,256]#LABEL
        
        x=x.astype('float32') 
        y=y.astype('int64') 
        x=torch.from_numpy(x);y=torch.from_numpy(y)
        x=x[:,np.newaxis,:,:]
        x = Variable(x,requires_grad=True);
#         x=x.cuda(0);y=y.cuda(0)
        self.x_data = x
        self.y_data = y
        self.len = x.shape[0] 
        
    def __getitem__(self, index):   
        return self.x_data[index], self.y_data[index]
 
    def __len__(self): #可返回数据条数
        return self.len

class myDataset(Dataset):#
    def __init__(self,mix,m_h,sig_num,seq_len,input_size):
        #路径由实例对象提供
        #  x0:INPUT     x1:MASK OF HS   x2：MASK OF LS
        x0=np.loadtxt(open(mix,"rb"),delimiter=",",skiprows=0)
        tempt=np.zeros([sig_num,seq_len,input_size]) 
        for i in range(sig_num):  
            for hang in range(seq_len):
                for lie in range(input_size):
                    tempt[i,hang,lie]=x0[hang+i*seq_len,lie]
        x0=tempt.astype('float32')
        
        x1=np.loadtxt(open(m_h,"rb"),delimiter=",",skiprows=0)
        tempt=np.zeros([sig_num,seq_len,input_size]) 
        for i in range(sig_num):  
            for hang in range(seq_len):
                for lie in range(input_size):
                    tempt[i,hang,lie]=x1[hang+i*seq_len,lie]
        x1=tempt.astype('float32')    
        
        x0=torch.from_numpy(x0);x1=torch.from_numpy(x1);

        x0=Variable(x0,requires_grad=True);x1=Variable(x1,requires_grad=True);
        
        x0=x0[:,np.newaxis,:,:];x1=x1[:,np.newaxis,:,:];
        x0.cuda(0);x1.cuda(0);

        self.len = x0.shape[0] # shape
        self.x0_data=x0;self.x1_data=x1;
        
   
    def __getitem__(self, index):    
        
        return self.x0_data[index].cuda(0),self.x1_data[index].cuda(0)
    
    def __len__(self): #
        return self.len
      

In [3]:
#--------------------2 MODELS----------------------
class MyDCNN(torch.nn.Module):
    def __init__(self):
        super(MyDCNN, self).__init__()
        self.pooling1=torch.nn.AvgPool2d(kernel_size=2,stride=2)
        self.cnn10=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,
                              stride=1,padding=1,bias=False, dilation=2)
    
        self.cnn20=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,
                              stride=1,padding=1,bias=False, dilation=2)

        self.cnn30=nn.Conv2d(in_channels=32,out_channels=48,kernel_size=3,
                        stride=1,padding=1,bias=False, dilation=2)
        
        self.cnn40=nn.Conv2d(in_channels=48,out_channels=64,kernel_size=3,
                        stride=1,padding=1,bias=False, dilation=2)
        self.bn4=nn.BatchNorm2d(64)
        self.fc1=torch.nn.Linear(5376,128)
        self.fc2=torch.nn.Linear(128,64)
        self.fc3=torch.nn.Linear(64,8)
    def forward(self,x):
        batch_size=x.size(0)
        en=nn.functional.elu(x, alpha=1.0, inplace=False)
        en=torch.relu(self.cnn10(en))
        en =self.pooling1(en)##############    
        en=torch.relu(self.cnn20(en))
        en =self.pooling1(en)##############
        en=torch.relu(self.cnn30(en))
        en =self.pooling1(en)##############
        en=torch.relu(self.cnn40(en))
        en=self.bn4(en)
        en =self.pooling1(en)##############
        en=en.view(batch_size,-1)
        en=self.fc1(en)
        en=self.fc2(en)
        en=self.fc3(en)
        return en
   

class MyUnet(nn.Module):
    def __init__(self):
        super().__init__()
        #CODING（3+1）
        self.bn1= nn.BatchNorm2d(48);self.bn2= nn.BatchNorm2d(96);self.bn3= nn.BatchNorm2d(144);
        self.bn0= nn.BatchNorm2d(1);
        self.encode1_1 =nn.Conv2d(1,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode1_2=nn.Conv2d(48,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode2_1=nn.Conv2d(48,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode2_2=nn.Conv2d(96,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode3_1=nn.Conv2d(96,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode3_2=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode4_1=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode4_2=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        #DECODING（3+1）
        self.upsample3= nn.ConvTranspose2d(144,144,kernel_size=2,stride=2,padding=0);                                       
        self.decode3_1 =nn.Conv2d(288,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode3_2 =nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.upsample2= nn.ConvTranspose2d(144,144,kernel_size=2,stride=2,padding=0);
        self.decode2_1 =nn.Conv2d(240,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode2_2 =nn.Conv2d(96,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.upsample1= nn.ConvTranspose2d(96,96,kernel_size=2,stride=2,padding=0);
        self.decode1_1 =nn.Conv2d(144,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode1_2 =nn.Conv2d(48,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode0=nn.Conv2d(48,1,kernel_size=3,stride=1,padding=1,bias=False)
    def forward(self,x):
        enc1 = torch.relu(self.encode1_1(x))
        enc1 = torch.relu(self.encode1_2(enc1))
        enc1 =self.bn1(enc1)
        enc2 = F.max_pool2d(enc1, kernel_size=2, stride=2, padding=0)
        enc2 = torch.relu(self.encode2_1(enc2))
        enc2 = torch.relu(self.encode2_2(enc2))
        enc2 =self.bn2(enc2)
        enc3 = F.max_pool2d(enc2, kernel_size=2, stride=2, padding=0)
        enc3 = torch.relu(self.encode3_1(enc3))
        enc3 = torch.relu(self.encode3_2(enc3))
        enc3 =self.bn3(enc3)
        enc4 = F.max_pool2d(enc3, kernel_size=2, stride=2, padding=0)       
        enc4 = torch.relu(self.encode4_1(enc4))
        enc4 = torch.relu(self.encode4_2(enc4))
        enc4 =self.bn3(enc4)
        #解码部分
        dec = self.upsample3(enc4)
        dec = torch.relu(self.decode3_1(torch.cat([enc3,dec],dim=1)))
        dec = torch.relu(self.decode3_2(dec))
        dec =self.bn3(dec)
        dec = self.upsample2(dec)
        dec = torch.relu(self.decode2_1(torch.cat([enc2,dec],dim=1)))
        dec = torch.relu(self.decode2_2(dec))
        dec =self.bn2(dec)
        dec = self.upsample1(dec)
        dec = torch.relu(self.decode1_1(torch.cat([enc1,dec],dim=1)))
        dec = torch.relu(self.decode1_2(dec))
        dec =self.bn1(dec)
        out = self.bn0(torch.tanh(self.decode0(dec)))
        return out


In [4]:
#----------------LOAD MODEL---------------
m_state_dict = torch.load('my_dcnn.pt')  #LOAD PARAS
new_m = MyDCNN()              
new_m.load_state_dict(m_state_dict) 

m_state_dict = torch.load('my_unet_0鼾声.pt')  
new_u =MyUnet()             
new_u.load_state_dict(m_state_dict) 
new_u.cuda()


MyUnet(
  (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encode1_1): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode1_2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode2_1): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode2_2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode3_1): Conv2d(96, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode3_2): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode4_1): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(

In [6]:
#--------------INPUT DCNN FEATURE-------------------
acc_all=[]
in_width=256
in_height=128
sig_num=250
dataset=lungDataset(inputs="C:\\Users\\HS\\Desktop\\DC+U\\测试数据\\dcnn_ltf_250_2肺泡呼吸音.csv",
                    sig_num=sig_num,in_width=in_width,
                    in_height=in_height)
dcnn_validation_loader=DataLoader(dataset=dataset,batch_size=sig_num,shuffle=False,num_workers=0)

# dataset=lungDataset(inputs="C:\\Users\\HS\\Desktop\\分类肺音\\DCNN训练数据_2240肺音分类时频矩阵.csv",
#                     sig_num=sig_num,in_width=in_width,
#                     in_height=in_height)
# dcnn_validation_loader=DataLoader(dataset=dataset,batch_size=sig_num,shuffle=True,num_workers=0)

#-----------DCNN------------
for step,(x,y) in enumerate(dcnn_validation_loader):
    train_corrects=0
    output_test=new_m(x)
    pre_lab=torch.argmax(output_test,1)
    print('预测分类',pre_lab);print('真实分类',y)
    corrects=torch.sum(pre_lab == y.data) #ACC pre_lab == b_y.data
    num=x.size(0) #
    acc_all.append(corrects.double().item()/num)
    print(acc_all)
    print('-'*50)

预测分类 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
真实分类 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [11]:
##-------------定义损失函数-----------------
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,out_pred,mix,m_h):
        lossh = mix* (out_pred - m_h)           #[[B, C, H, W]]
        loss=torch.sum(torch.sum(torch.pow(lossh, 2), 1))
        return loss
criterion = My_loss()
#--------------unet-------------------
seq_len=256         
input_size=128      
sig_num=400
dataset=myDataset(mix="C:\\Users\\HS\\Desktop\\DC+U\\测试数据\\unet_mix_400_4哮鸣音.csv",
                  m_h="C:\\Users\\HS\\Desktop\\DC+U\\测试数据\\unet_m_h_400_4哮鸣音.csv",
                  sig_num=sig_num,seq_len=seq_len,input_size=input_size)
unet_valid_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False,num_workers=0) 
    
#-----------unet------------ 
for index, (mix,m_h) in enumerate(unet_valid_loader,0):
    if pre_lab.data[index]==0:
        m_state_dict = torch.load('my_unet0_鼾声.pt')  
    if pre_lab.data[index]==1:
        m_state_dict = torch.load('my_unet01_湿罗音.pt')   
    if pre_lab.data[index]==2:
        m_state_dict = torch.load('my_unet2_肺泡呼吸音.pt')  
    if pre_lab.data[index]==3:
        m_state_dict = torch.load('my_unet3_支气管呼吸音.pt')   
    if pre_lab.data[index]==4:
        m_state_dict = torch.load('my_unet4_哮鸣音.pt')    

    new_u =MyUnet()              
    new_u.load_state_dict(m_state_dict) 
    new_u.cuda()
    
    out_pred=new_u(mix)
    loss = criterion(out_pred,mix,m_h)
    print('第{:3d}个信号的loss: {:5.2f}'.format(index+1,loss))
    # -----------------PREDICT MASK-------------
    out_h=out_pred;out_l=1-out_pred
    h_mask=torch.reshape(out_h,(256,128));l_mask=torch.reshape(out_l,(256,128))

    h_mask=h_mask.cpu();l_mask=l_mask.cpu()
    h_mask=h_mask.detach().numpy();l_mask=l_mask.detach().numpy()
    np.savetxt('C:\\Users\\HS\Desktop\\DC+U\\unet生成数据\\400_4哮鸣音h_mask%d.csv'%(index+1), h_mask, delimiter = ',')
    np.savetxt('C:\\Users\\HS\\Desktop\\DC+U\\unet生成数据\\400_4哮鸣音l_mask%d.csv'%(index+1), l_mask, delimiter = ',')
#     print(h_mask.shape);print(l_mask.shape)
    # -----------------OUTPUT :MASKS OF HS AND LS-------------
    

第  1个信号的loss: 428.17
第  2个信号的loss: 1111.42
第  3个信号的loss: 421.05
第  4个信号的loss: 476.24
第  5个信号的loss: 325.42
第  6个信号的loss: 614.00
第  7个信号的loss: 176.94
第  8个信号的loss: 221.32
第  9个信号的loss: 484.40
第 10个信号的loss: 917.24
第 11个信号的loss: 339.54
第 12个信号的loss: 262.80
第 13个信号的loss: 321.26
第 14个信号的loss: 684.62
第 15个信号的loss: 444.37
第 16个信号的loss: 306.14
第 17个信号的loss: 281.38
第 18个信号的loss: 228.60
第 19个信号的loss: 949.51
第 20个信号的loss: 885.14
第 21个信号的loss: 542.04
第 22个信号的loss: 1018.69
第 23个信号的loss: 283.06
第 24个信号的loss: 552.12
第 25个信号的loss: 405.07
第 26个信号的loss: 486.73
第 27个信号的loss: 206.32
第 28个信号的loss: 276.89
第 29个信号的loss: 599.57
第 30个信号的loss: 400.67
第 31个信号的loss: 3270.92
第 32个信号的loss: 310.12
第 33个信号的loss: 445.86
第 34个信号的loss: 585.93
第 35个信号的loss: 274.19
第 36个信号的loss: 195.52
第 37个信号的loss: 240.02
第 38个信号的loss: 240.18
第 39个信号的loss: 467.29
第 40个信号的loss: 225.69
第 41个信号的loss: 2974.68
第 42个信号的loss: 300.00
第 43个信号的loss: 486.58
第 44个信号的loss: 1445.80
第 45个信号的loss: 453.81
第 46个信号的loss: 414.59
第 47个信号的loss: 3038.27
第 48个信号

第397个信号的loss: 3785.19
第398个信号的loss: 1431.33
第399个信号的loss: 1615.74
第400个信号的loss: 954.14
